In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer,pipeline,AutoModelForTokenClassification

###Load NER Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

In [ ]:
pipe=pipeline('ner',model=model,tokenizer=tokenizer,aggregation_strategy='simple')

In [ ]:
ner_output=pipe("body hot and pain in chest")
print(ner_output)

NameError: name 'pipe' is not defined

In [ ]:
symptoms = []
biological_structure = None
combined_symptoms = []
for entity in ner_output:
    if entity['entity_group'] == 'Biological_structure':
        biological_structure = entity['word']
    elif entity['entity_group'] == 'Sign_symptom':
        symptom = entity['word']
        if biological_structure:
            symptoms.append(f'"{biological_structure} {symptom}"')
            biological_structure = None
        else:
            symptoms.append(f'"{symptom}"')

# Join symptoms into a single string
symptoms_str = ', '.join(symptoms)

In [ ]:
symptoms_str

'"body pain"'

###Language Classification

In [ ]:
import joblib
model_langid = joblib.load('language_classifier_ngram_2_6.joblib')
vectorizer_langid = joblib.load('vectorizer.joblib')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
#create pipeline for language classification
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
pipeline_trans = Pipeline([
    ('vectorizer', vectorizer_langid),
    ('classifier', model_langid)
])

In [ ]:
!pip install translate
!pip install indic-transliteration

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.5 MB/s eta 0:00:00


###Phrase Segmentation and Translation

In [ ]:
from googletrans import Translator

In [ ]:
from translate import Translator
from indic_transliteration import sanscript
#identify language
def identify_language(word):
    #print(word)
    prediction=pipeline_trans.predict([word])
    if prediction[0]==0:
        return 'en'
    else:
        return 'hi'
#translate phrase to hindi
def translate_to_hindi(word):
    #print("A: ",word)
    translator = Translator(from_lang="en",to_lang="hi")
    translation = translator.translate(word)
    return translation
#transliterate phrase to hindi
def transliterate_to_devanagari(word):
    transliterated_word = sanscript.transliterate(word, sanscript.ITRANS, sanscript.DEVANAGARI)
    return transliterated_word
#translate sentence to english
def translate_to_english(sentence):
   # print(sentence)
    translator = Translator(from_lang="hi",to_lang="en")
    translation = translator.translate(sentence)
    print(translation)
    return translation
#phrase segmentation without aggregation and function calls without keyword replacement
def translate_hinglish(sentences):
    translated_sentences = []
    for sentence in sentences:
        words = sentence.split()
        output=""

        for word in words:
            language = identify_language(word)
            print(word,language)
            if language == 'en':
                translated_word = translate_to_hindi(word)
                output+=translated_word+" "
            elif language == 'hi':
                transliterated_word = transliterate_to_devanagari(word)
                output+=transliterated_word+" "
        op2=translate_to_english(output)
        translated_sentences.append(op2)
    return translated_sentences




In [ ]:
#phrase segmentation with aggregation and function calls without keyword replacement. Performs better with aggregation for contextual sentences
def translate_hinglish(sentences):
    translated_sentences = []
    for sentence in sentences:
        words = sentence.split()
        output = ""
        start = 0
        end = 0

        while start < len(words):
            current_language = identify_language(words[start])
            end = start + 1
            while end < len(words) and identify_language(words[end]) == current_language:
                end += 1
            current_sequence = ' '.join(words[start:end])

            if current_language == 'en':
                translated_sequence = translate_to_hindi(current_sequence)
            else:
                translated_sequence = transliterate_to_devanagari(current_sequence)

            output += translated_sequence + " "
            start = end

        translated_sentence = translate_to_english(output.strip())
        translated_sentences.append(translated_sentence)

    return translated_sentences


In [ ]:
#phrase segmentation with aggregation and function calls with keyword replacement
def translate_hinglish_2(sentences):
    translated_sentences = []
    for sentence in sentences:
        sentence=sentence.lower()
        nwords = sentence.split()
        words=[]
        for i, w in enumerate(nwords):
            try:
                if w in hindi_dictionary:
                    words.append((hindi_dictionary[w],"en"))
                else:
                     words.append((w,identify_language(w)))
            except KeyError:
                pass
        output = ""
        start = 0
        end = 0

        while start < len(words):

            current_language = words[start][1]
            end = start + 1
            while end < len(words) and (words[end][1]) == current_language:
                end += 1
            current_sequence = ' '.join(word[0] for word in words[start:end])

            if current_language == 'en':
                translated_sequence = translate_to_hindi(current_sequence)
            else:
                translated_sequence = transliterate_to_devanagari(current_sequence)

            output += translated_sequence + " "
            start = end

        translated_sentence = translate_to_english(output.strip())
        translated_sentences.append(translated_sentence)

    return translated_sentences


In [ ]:
#medical keywords and their
hindi_dictionary = {
    'bukhar':'fever',
    'khansi': 'Cough',
    'daura': 'Attack',
    'kaf': 'Phlegm',
    'tanav': 'Stress',
    'krodh': 'Anger',
    'bhay': 'Fear',
    'jukam': 'Cold',
    'najla': 'Runny nose',
    'jalan': 'Burning sensation',
    'gard': 'Filth',
    'gandagi': 'Dirtiness',
    'sardi': 'Cold',
    'flu': 'Flu',
    'garam': 'fever',
    'thandi': 'cold',
    'pyaas': 'Thirst',
    'jakhm': 'Wound',
    'sankraman': 'Infection',
    'thakan': 'Fatigue',
    'jodo': 'joints',
    'dard': 'Pain',
    'sir': 'Head',
    'sar' : 'Head',
    'cheenkne': 'Sneezing',
    'gala': 'throat',
    'kharaab':'bad',
    'laali': 'Redness',
    'laal':'Redness',
    'aankh':'eye',
    'khujli': 'Itching',
    'naak': 'nose',
    'tootne': 'Breaking',
    'thanda': 'Cold',
    'chehra': 'face',
    'pet': 'Stomach',
    'saans': 'Breath',
    'chaati': 'Chest',
    'ulti': 'Vomiting',
    'dakar': 'Burping',
    'matli': 'Nausea',
    'bechaini': 'Restlessness',
    'jalan':'burning',
    'chaati':'chest',
    'seena':'chest',
    'seene':'chest',
    'dane':'spots',
    'sharir':'body',
    'chhikh':'sneezing',
    'thakaan':'tiredness',
    'bhaari':'heavy',
    'bathroom':'urination',
    'sard':'cold',
    'kharash':'irritation',
    'gala':'throat',
    'gale':'throat',
    'khujli':'itching',
    'aankh':'eyes',
    'aankhon':'eyes',
    'chakkar':'dizziness',

}


###Disease Diagnosis using Fine-Tuned BERT

In [ ]:
#Load fine-tuned BERT model
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import TextClassificationPipeline
# Load the model
model_diagnosis = TFAutoModelForSequenceClassification.from_pretrained('/content/')

# Load the tokenizer
tokenizer_diagnosis = AutoTokenizer.from_pretrained('/content/')
pipe_diagnosis = TextClassificationPipeline(model=model_diagnosis, tokenizer=tokenizer_diagnosis, top_k = 24)

Some layers from the model checkpoint at /content/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/translated_symptoms_3.csv')

In [ ]:
df['translated_symptoms'] = df['Symptom Description'].apply(lambda x: translate_hinglish_2([x])[0])

For a few days, the skin is becoming very dry patches, and they become itchy. There are white patches on the skin.
The eyes are yellow and the fatigue is gone.
Paroneme veins are very visible and cause pain when ejaculating. Kabhi Kabhi Kabhi Par is being lightly burned is Mehsus Hoti.
On the main part, the little spots have come out, which I felt very much that you are in your office party. The small spots burst and the water comes out.
There are small spots on the face that are sometimes painful.
Garden mein is a sharp pain and the shoulders are also stiff.
Paroneme veins are very visible and cause pain when ejaculating. The pain is severe and lasts for a long time.
By taking the drug, red marks are formed on the skin and there is a light swelling.
On the main part, the little spots have come out, which I felt very much that you are in your office party. The body is heavy with fever.
Nasal congestion and sneezing are common.
There are small red lesions on the face that sometimes leak

In [ ]:
df['translated_symptoms2'] = df['Symptom Description'].apply(lambda x: translate_hinglish([x])[0])

The skin is becoming very dry patches, and they are itchy. There are white patches on the skin.
The bell rings, and he is exhausted.
Iron ore wines are highly visible and suffer from erectile dysfunction. Some of the pairs are light watery.
There are a lot of kids out there who are very fussy. The water evaporates, and the water evaporates.
There are a lot of kids who are in pain.
The neck is sharp and the shoulders are hard.
Iron ore wines are highly visible and suffer from erectile dysfunction. The pain lasts for a long time and lasts for a long time.
There are red marks on the skin and the hulk is swollen.
There are a lot of kids out there who are very fussy. The body is loaded with bubbles.
There is more to life than sorrow, and there is more to life than sorrow.
There are small red lesions on the face that sometimes leak.
Frequent bathrooms are familiar, and cafes are exhausting.
There has been no fever for a few days now, and the petnabhi coffee is getting tighter. The pineapple 

In [ ]:
df['final_diagnosis'] = ''  # Create an empty column to store the final diagnosis
#initialize metrics for performance calculation
count=0
count2=0
p1=0
p2=0
p3=0
p4=0
p5=0
for index, row in df.iterrows():
  symptom_description = row['translated_symptoms']
  diagnosis = pipe_diagnosis(symptom_description)

  try:
    final_diagnosis = diagnosis[0][0]['label'] + " or " + diagnosis[0][1]['label'] + " or " + diagnosis[0][2]['label'] + " or " + diagnosis[0][3]['label'] + " or " + diagnosis[0][4]['label']
  except IndexError:
    # Handle cases where there are fewer than 3 predictions
    final_diagnosis = "Diagnosis not available"

  df.loc[index, 'final_diagnosis'] = final_diagnosis # Store the diagnosis in the new column
  print(f"Diagnosis: {final_diagnosis}")
  print(f"Original Diagnosis: {row['Diagnosis']}")
  if row['Diagnosis'] in final_diagnosis:
    count2+=1
  if row['Diagnosis'] in diagnosis[0][0]['label']:
    count+=3
    p1+=1
  if row['Diagnosis'] in diagnosis[0][1]['label']:
    count+=2
    p2+=1
  if row['Diagnosis'] in diagnosis[0][2]['label']:
    count+=1
    p3+=1
  if row['Diagnosis'] in diagnosis[0][3]['label']:
    count+=1
    p4+=1
  if row['Diagnosis'] in diagnosis[0][4]['label']:
    count+=1
    p5+=1
print(count)
print(count2)
print(p1,p2,p3,p4,p5)

Diagnosis: Fungal infection or Psoriasis or Chicken pox or Impetigo or Acne
Original Diagnosis: Psoriasis
Diagnosis: Psoriasis or Dengue or Chicken pox or peptic ulcer disease or allergy
Original Diagnosis: Jaundice
Diagnosis: Varicose Veins or Chicken pox or peptic ulcer disease or Psoriasis or diabetes
Original Diagnosis: Varicose Veins
Diagnosis: Fungal infection or Chicken pox or Impetigo or Acne or Dimorphic Hemorrhoids
Original Diagnosis: Chicken pox
Diagnosis: Chicken pox or Psoriasis or Fungal infection or Dengue or Varicose Veins
Original Diagnosis: Acne
Diagnosis: Arthritis or Varicose Veins or Psoriasis or Cervical spondylosis or peptic ulcer disease
Original Diagnosis: Cervical spondylosis
Diagnosis: Varicose Veins or Chicken pox or Psoriasis or diabetes or peptic ulcer disease
Original Diagnosis: Varicose Veins
Diagnosis: Chicken pox or Dengue or Psoriasis or Fungal infection or Impetigo
Original Diagnosis: drug reaction
Diagnosis: Chicken pox or Fungal infection or Dengue

In [ ]:
df['final_diagnosis2'] = ''  # Create an empty column to store the final diagnosis
#initialize metrics for performance calculation
count=0
count2=0
p1=0
p2=0
p3=0
p4=0
p5=0
for index, row in df.iterrows():
  symptom_description = row['translated_symptoms2']
  diagnosis = pipe_diagnosis(symptom_description)

  try:
    final_diagnosis = diagnosis[0][0]['label'] + " or " + diagnosis[0][1]['label'] + " or " + diagnosis[0][2]['label'] + " or " + diagnosis[0][3]['label'] + " or " + diagnosis[0][4]['label']
  except IndexError:
    # Handle cases where there are fewer than 3 predictions
    final_diagnosis = "Diagnosis not available"

  df.loc[index, 'final_diagnosis2'] = final_diagnosis # Store the diagnosis in the new column
  print(f"Diagnosis: {final_diagnosis}")
  print(f"Original Diagnosis: {row['Diagnosis']}")
  if row['Diagnosis'] in final_diagnosis:
    count2+=1
  if row['Diagnosis'] in diagnosis[0][0]['label']:
    count+=3
    p1+=1
  if row['Diagnosis'] in diagnosis[0][1]['label']:
    count+=2
    p2+=1
  if row['Diagnosis'] in diagnosis[0][2]['label']:
    count+=1
    p3+=1
  if row['Diagnosis'] in diagnosis[0][3]['label']:
    count+=1
    p4+=1
  if row['Diagnosis'] in diagnosis[0][4]['label']:
    count+=1
    p5+=1
print(count)
print(count2)
print(p1,p2,p3,p4,p5)

Diagnosis: Fungal infection or Psoriasis or Chicken pox or Impetigo or peptic ulcer disease
Original Diagnosis: Psoriasis
Diagnosis: Dimorphic Hemorrhoids or diabetes or peptic ulcer disease or Varicose Veins or Chicken pox
Original Diagnosis: Jaundice
Diagnosis: peptic ulcer disease or Psoriasis or diabetes or gastroesophageal reflux disease or Fungal infection
Original Diagnosis: Varicose Veins
Diagnosis: Pneumonia or Bronchial Asthma or Fungal infection or peptic ulcer disease or urinary tract infection
Original Diagnosis: Chicken pox
Diagnosis: Impetigo or Chicken pox or Fungal infection or urinary tract infection or Typhoid
Original Diagnosis: Acne
Diagnosis: Psoriasis or Varicose Veins or allergy or Arthritis or diabetes
Original Diagnosis: Cervical spondylosis
Diagnosis: Psoriasis or peptic ulcer disease or gastroesophageal reflux disease or Varicose Veins or Chicken pox
Original Diagnosis: Varicose Veins
Diagnosis: Fungal infection or Psoriasis or Acne or Chicken pox or Varicos

In [ ]:
#diagnosis
for index, row in df.iterrows():
  symptom_description = row['translated_symptoms']
  diagnosis = pipe_diagnosis(symptoms_str)
  final_diagnosis = diagnosis[0][0]['label'] + " or " + diagnosis[0][1]['label'] + " or " + diagnosis[0][2]['label']
  print(f"Diagnosis ': {final_diagnosis}")
  print(f"Original Diagnosis: {row['Diagnosis']}")

In [ ]:
from transformers import TextClassificationPipeline

pipe_diagnosis = TextClassificationPipeline(model=model_diagnosis, tokenizer=tokenizer_diagnosis, top_k = 24)
pred2 = pipe_diagnosis("I have constipation and belly pain, and it's been really uncomfortable. The belly pain has been getting worse and is starting to affect my daily life. Moreover, I get chills every night, followed by a mild fever.")
print(pred2[0][0]['label'])

Typhoid


In [ ]:
translate_hinglish_2(["Mujhe bukhar aa raha hai chaati mein dard ho raha hai and khaansi aa rahi hai"])

I'm having a fever, chest pain and coughing


["I'm having a fever, chest pain and coughing"]

In [ ]:
# create a pipe line for multiple ML models, where the input is a string. first it is translated using the translate_hinglish_2 function, then the output is sent to 'pipe' for NER and the result is printed

from sklearn.pipeline import Pipeline

def process_text(text):
  translated_text = translate_hinglish_2([text])[0]
  ner_output = pipe(translated_text)
  #print(ner_output)
  symptoms = []
  biological_structure = None
  combined_symptoms = []
  for entity in ner_output:
      if entity['entity_group'] == 'Biological_structure':
          biological_structure = entity['word']
      elif entity['entity_group'] == 'Sign_symptom':
          symptom = entity['word']
          if biological_structure:
              symptoms.append(f'"{biological_structure} {symptom}"')
              biological_structure = None
          else:
              symptoms.append(f'"{symptom}"')

  # Join symptoms into a single string
  symptoms_str = ', '.join(symptoms)
  #print(symptoms_str)
  diagnosis = pipe_diagnosis(symptoms_str)
  final_diagnosis = diagnosis[0][0]['label'] + " or " + diagnosis[0][1]['label'] + " or " + diagnosis[0][2]['label']
  return final_diagnosis



# Example usage
input_text = "Mujhe ulti aa rahi hai, fever aa raha hai aur khaansi bhi"
process_text(input_text)
input_text = "Mujhe sir dard ho raha hai, and khaansi aa rahi hai. Chest pain bhi ho raha hai"
process_text(input_text)


I am having a vomiting, fever and cough
I'm having a headache, and coughing. Chest pain


'Hypertension or Dengue or allergy'

In [ ]:
#download df as csv

df.to_csv('translated_symptoms.csv', index=False)

In [ ]:
#row-wise iteration fo diagnosis
for index, row in df.iterrows():
  symptom_description = row['Symptom Description']
  diagnosis_result = process_text(symptom_description)
  print(f"Diagnosis for '{symptom_description}': {diagnosis_result}")
  print(f"Original Diagnosis: {row['Diagnosis']}")

For a few days, the skin is becoming very dry patches, and they become itchy. There are white patches on the skin.
Diagnosis for 'Kuch dino se skin mein bahut dry patches ban rahe hain, aur wo itchy lagte hain. Skin pe kabhi kabhi white flakes dikhte hain.': drug reaction or diabetes or allergy
Original Diagnosis: Psoriasis
The eyebrows are pale and tired.
Diagnosis for 'Aankhon ka rang peela ho gaya hai, aur thakaan mehsoos hoti hai.': Psoriasis or gastroesophageal reflux disease or peptic ulcer disease
Original Diagnosis: Jaundice
Paroneme veins are very visible and cause pain when ejaculating. Kabhi Kabhi Kabhi Par is being lightly burned is Mehsus Hoti.
Diagnosis for 'Pairon mein veins bahut zyada dikh rahi hain aur khade hone pe dard hota hai. Kabhi kabhi pair halki jalan mehsoos hoti hai.': Psoriasis or peptic ulcer disease or Dengue
Original Diagnosis: Varicose Veins
Sharir Pe Chhote Chhote Dane has come out, which is very much what I felt that you are doing in your office party

In [ ]:
# prompt: df.to_csv

df.to_csv('translated_symptoms_3.csv', index=False)

In [ ]:
!pip install torch --upgrade --force-reinstall
!pip install transformers --upgrade --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 740.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==1.1.1
aiohappyeyeballs==2.4.4
aiohttp==3.11.10
aiosignal==1.3.1
alabaster==1.0.0
albucore==0.0.19
albumentations==1.4.20
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.20.0
astropy==6.1.7
astropy-iers-data==0.2024.12.9.0.36.21
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
backports.functools-lru-cache==2.0.0
beautifulsoup4==4.12.3
bigframes==1.27.0
bigquery-magics==0.4.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==2.7.1
bokeh==3.6.2
Bottleneck==1.4.2
bqplot==0.12.43
branca==0.8.0
CacheControl==0.14.1
cachetools==5.5.0
catalogue==2.0.10
certifi==2024.8.30
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.0
chex==0.1.87
clarabel==0.9.0
click==8.1.7
cloudpathlib==0.20.0
cloudpickle==3.1.0
cmake==3.30.5
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confe